In [69]:
import os
import glob
import pydicom
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2
import numpy as np
import matplotlib.patches as patches
from nms import nms
from PIL import Image

In [48]:
#!pip install nms

## Cascade

In [55]:
vessel_cascade = cv2.CascadeClassifier("cascade.xml")

In [56]:
def parse_filename(filename):
    n_id = filename.split(os.sep)[1]
    n_sl = filename.split(os.sep)[-1][4:8]
    return n_id, n_sl

In [57]:
dcm_files = glob.glob('Z:/Lisa/PARISK/PARISK-MRI-Carotid/MUMC*/MASSExport/T1w_*.dcm')
con_files = glob.glob('Z:/Lisa/PARISK/PARISK-MRI-Carotid/MUMC*/contours/con_*.dcm')

In [58]:
l = []

In [85]:
for dcm_file in dcm_files:
    
    patient_number, slice_number = parse_filename(dcm_file)
    
    dcm = pydicom.read_file(dcm_file, force = True)
    dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
    dcm_img = dcm.pixel_array
    
    dcm_min = np.min(dcm_img)
    dcm_max = np.max(dcm_img)
    
    dcm_res = (np.round((dcm_img-dcm_min)/(dcm_max-dcm_min)*255)).astype(np.uint8)
    
    vessels = vessel_cascade.detectMultiScale3(dcm_res, 1.1, 3, minSize=(20, 20), maxSize=(75, 75), outputRejectLevels=True)
    
    vessel_rects = []
    for vessel in vessels[0]:
        vessel_rects.append((vessel[0], vessel[1], vessel[2], vessel[3]))
    
    confs = []
    for c in vessels[2]:
        confs.append(c[0])
        
    idxs = nms.boxes(vessel_rects, confs)
    
    fig, ax = plt.subplots(figsize = (10,10)) 
    plt.xticks([])
    plt.yticks([])
    imgplot = plt.imshow(cv2.cvtColor(dcm_res, cv2.COLOR_BGR2RGB))

    for idx in idxs:
        x = vessels[0][idx][0]
        y = vessels[0][idx][1]
        w = vessels[0][idx][2]
        h = vessels[0][idx][3]
        rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, str(idx), horizontalalignment='left', verticalalignment='top', color='y')
        
        box = np.copy(dcm_res[y:y+h,x:x+w])
        im = Image.fromarray(box)
        im.save('Z:/lisa/vessel_class_data/'+ patient_number + '_' + slice_number + '_' + str(idx) + '.png')

    plt.title(patient_number + ' ' + slice_number)
    plt.savefig('Z:/Lisa/vessel_det_res/'+ patient_number + '_' + slice_number + '.png')
    plt.close()

## Save for segmentation

In [86]:
t1w_files = glob.glob('Z:/Lisa/PARISK/PARISK-MRI-Carotid/MUMC*/MASSExport/T1w_*.dcm')
tof_files = glob.glob('Z:/Lisa/PARISK/PARISK-MRI-Carotid/MUMC*/MASSExport/3DTOF*.dcm')
con_files = glob.glob('Z:/Lisa/PARISK/PARISK-MRI-Carotid/MUMC*/contours/con_*.dcm')

In [95]:
for i in range (0, len(con_files)):
    
    t1w_file = t1w_files[i]
    tof_file = tof_files[i]
    con_file = con_files[i]
    
    patient_number, slice_number = parse_filename(t1w_file)
    
    t1w_dcm = pydicom.read_file(t1w_file, force = True)
    t1w_dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
    t1w = t1w_dcm.pixel_array

    t1w_min = np.min(t1w)
    t1w_max = np.max(t1w)

    t1w_res = (np.round((t1w-t1w_min)/(t1w_max-t1w_min)*255)).astype(np.uint8)

    tof_dcm = pydicom.read_file(tof_file, force = True)
    tof_dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
    tof = tof_dcm.pixel_array

    con_dcm = pydicom.read_file(con_file, force = True)
    con_dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
    con = con_dcm.pixel_array

    fig, ax = plt.subplots(figsize = (20,10)) 

    plt.subplot(1, 2, 1)
    plt.imshow(t1w)
    plt.contour(con, levels=[0.5], colors='r')
    plt.axis('Off')

    plt.subplot(1, 2, 2)
    plt.imshow(tof)
    plt.axis('Off')

    plt.suptitle(patient_number + ' ' + slice_number)
    plt.savefig('Z:/Lisa/vessel_train/t1w_tof/'+ patient_number + '_' + slice_number + '.png')
    plt.close()

    im = Image.fromarray(t1w_res)
    im.save('Z:/lisa/vessel_train/t1w/'+ patient_number + '_' + slice_number +  '.png')